In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jun  2 11:09:55 2018

@author: Marcelo
"""

import numpy as np
import numpy.ma as ma  # masked array
from machine_selection import set_screen_size_by_machine

# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

# 2019-03-09
from teste_datas import (Spec)

# 2019-09-16
from gauss_funcs import gaus_fw, gaus_sig

# 2019-07-15
# https://docs.scipy.org/doc/numpy/reference/routines.polynomials.classes.html#basics
from numpy.polynomial import Polynomial as P
from numpy.polynomial.polynomial import (polyfit, polyval)

from scipy.signal import (cwt, ricker, find_peaks, find_peaks_cwt, peak_widths)
from scipy.ndimage import label, generate_binary_structure, find_objects  # 2019-09-18

from scipy.optimize import (curve_fit, root)
from scipy.stats import (iqr)
from scipy.special import (expit)
import base_line_funcs as blf
import spectra_regions_funcs as spreg

# from spectra_regions_funcs import (total_analysis2)
import csv
import pickle
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh import palettes

# 2020-02-21
from spectrum_analysis import cwt_complete_analysis

In [2]:
default_plot_width, default_plot_height = set_screen_size_by_machine()

In [11]:
8 in range(8,10)

True

In [3]:
# 2019-09-26 Importar calibra´c~ao em efici^encia de arquivo CSV 

In [4]:
effics = pd.read_csv('2019-sep-25 ALMERA Samples CSV.csv',decimal=',',header=None)

In [5]:
effics

,0,1,2
0,46.54,0.008256,0.026447
1,475.35,0.002822,0.045887
2,563.23,0.002291,0.013179
3,569.32,0.002218,0.010851
4,604.70,0.002068,0.006972
5,795.84,0.001672,0.007033
6,801.93,0.001743,0.013259
7,1038.57,0.001614,0.067092
8,1167.94,0.001424,0.037835
9,1365.15,0.001292,0.026036


In [6]:
# 2019-09-26 PAREY AKIH

In [7]:
# 2019-09-26 Uma hora usarei isso: ler LibreOffice via Pandas!
# pd.read_excel('path_to_file.ods', engine='odf')

In [8]:
# constante k: ver no Maxima: (sqrt(3)*6^(3/2))/(2^(3/2)*5^(5/4)*%pi^(1/4));
k = (np.sqrt(3)*6**(3/2))/(2**(3/2)*5**(5/4)*np.pi**(1/4))
k

0.9041542662259198

In [9]:
kfwhm = 2*np.sqrt(2*np.log(2))
kfwhm

2.3548200450309493

In [10]:
def searchcwt(y0s, ipks, widths):
    widthmaxes = np.ones_like(ipks,dtype=float)
    sigmas_experim = np.ones_like(ipks,dtype=float)
    gau_hights = np.ones_like(ipks,dtype=float)
    k = (np.sqrt(3)*6**(3/2))/(2**(3/2)*5**(5/4)*np.pi**(1/4))
    for enumpk, ipk in enumerate(ipks):
        print('pico no canal ', ipk)
        if (ipk>25) & (ipk<len(y0s)-35):
            insca=1
            fisca=len(widths)+1
            newwidths=widths
            for j in range(1,4):
                print ('larguras:')
                print(newwidths)
                cwtmatr = cwt(y0s, ricker, newwidths)
                # 2019-07-15 AQUI: defini janela ao redor do pico
                # para analisar resultados de cwtmatr
                max_escalas = np.max(cwtmatr[:,ipk-10:ipk+11],axis=1)
                print ('max escalas:')
                print(max_escalas)
                imax = np.argmax(max_escalas)
                print('imax:')
                print(imax)
                i0 = imax-3
                if i0 < 0:
                    i0 = 0
                insca = newwidths[i0]
                i1 = imax+3
                if i1 >= len(newwidths):
                    i1 = len(newwidths)-1
                fisca = newwidths[i1]
                newwidths = np.linspace(insca,fisca,num=30)
            widthmaxes[enumpk] = newwidths[imax]
            print('widthmax:')
            print(widthmaxes[enumpk])
            print('------')
            sigmas_experim[enumpk] = widthmaxes[enumpk] / np.sqrt(5)
            gau_hights[enumpk] = k * max_escalas[imax] / np.sqrt(sigmas_experim[enumpk])
            print('gauss_height:')
            print(gau_hights[enumpk])
            print('sigma_experim:')
            print(sigmas_experim[enumpk])
            print('*******')
        else:
            print('pico extremo')
    return sigmas_experim, gau_hights

In [11]:
def searchcwt2(y0s, ipks, widths):
    widthmaxes = np.ones_like(ipks,dtype=float)
    sigmas_experim = np.ones_like(ipks,dtype=float)
    gau_hights = np.ones_like(ipks,dtype=float)
    k = (np.sqrt(3)*6**(3/2))/(2**(3/2)*5**(5/4)*np.pi**(1/4))
    cwtmatr2 = cwt(y0s, ricker, widths)
    return cwtmatr2

In [12]:
# 2019-07-23:
# Busca cwt usando escala movel obtida por ajuste fwhm x canal
# O polinomio de ajuste fit eh passado como parahmetro 
def searchcwt_fix(y0s):
    ipks_fix = []
    for i in range(20, len(y0s)-20):
        fff = find_peaks_cwt(y0s[i-10:i+11], [5.0])
    ipks_fix.append(fff)
    return ipks_fix

In [13]:
fn2 = '../gamma/spectra/ALMERA-2017-3/ALM-17-3-#5_PreliA.IEC'
spec2 = Spec()
if spec2.readSp( fn2 ) >= 0:
    print (spec2.spLngData)

8192


In [14]:
counts = spec2.spCounts
len(counts)

8192

In [15]:
ar1, xpk, ypk, peak_constructed_spectrum, bl_median_smoother = cwt_complete_analysis(counts)

(49, 14)
(0, 0)
(6, 1)
(6, 1)
(2, 1)
(4, 5)
(0, 1)
(7, 0)
(5, 0)
(27, 0)
(0, 1)
(4, 1)
(1, 0)
(3, 0)
(2, 0)
(0, 1)
(0, 0)
(0, 0)
(5, 1)
(0, 1)
(4, 1)
(8, 1)
(5, 0)
(2, 1)
(0, 0)
(0, 0)
(6, 3)
(9, 1)
(6, 1)
(3, 0)
(10, 1)
(0, 0)
(9, 0)
(0, 0)
(1, 0)
(12, 1)
(0, 0)
(3, 0)
(27, 1)
(0, 1)
(0, 0)
(10, 1)
(0, 0)
(0, 0)
(11, 1)
(5, 0)
(0, 0)
(0, 1)
(21, 3)
(1, 0)
(22, 1)
(0, 0)
(12, 1)
(6, 1)
(14, 2)
(17, 1)
(0, 0)
(35, 1)
(3, 0)
(14, 1)
(0, 0)
(0, 1)
(1, 0)
(7, 1)
(10, 1)
(15, 1)
(1, 0)
(0, 1)
(16, 0)
(0, 1)
(0, 0)
(5, 1)
(0, 0)
(3, 1)
(20, 1)
(24, 1)
(0, 1)
(10, 1)
(3, 0)
(19, 0)
(0, 0)
(19, 1)
(21, 1)
(18, 0)
(5, 1)
(7, 2)
(0, 0)
(21, 1)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(3, 0)
(0, 0)
(0, 0)
(0, 1)
(0, 0)
(1, 0)
(0, 0)
(20, 1)
(0, 1)
(0, 0)
(21, 1)
(2, 1)
(0, 0)
(0, 0)
(15, 1)
(12, 2)
(3, 0)
(0, 0)
(0, 0)
(1, 0)
(2, 0)
(0, 0)
(2, 1)
(0, 1)
(7, 1)
(7, 0)
(2, 1)
(0, 0)
(1, 0)
(1, 0)
(2, 1)
(23, 1)
(1, 0)
(0, 0)
(0, 1)
(0, 1)
(49, 1)
(0, 0)
(4, 1)
(2, 1)
(0, 0)
(1, 0)
(14, 1)
(0, 0)
(23, 1)
(0, 0)


In [17]:
# Bokeh graph w/ peaks in all scales
bokridges = figure(
   plot_width =default_plot_width,
   plot_height=int(default_plot_height*0.4),
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="synth",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokridges.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
# bokridges.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokridges)

In [18]:
xs = np.linspace(0,spec2.spLngData-1,spec2.spLngData)
y0s = np.asarray(spec2.spCounts)

In [19]:
# Bokeh graph w/ peaks in all scales
bokpksallscl = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
    y_axis_type="log",
    y_range=[10**1, 10**5], title="real spec",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
# bokpksallscl.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
bokpksallscl.line(xs, y0s, color="green")
# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokpksallscl)

In [20]:
# Bokeh graph w/ peaks in all scales
bokpksallscl = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
    y_axis_type="log",
    y_range=[10**1, 10**5], title="real spec",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
# bokpksallscl.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
bokpksallscl.line(xs, y0s, color="green")
bokpksallscl.line(xs, peak_constructed_spectrum, color="red")
bokpksallscl.line(xs, bl_median_smoother, color="cyan")
bokpksallscl.line(xs, peak_constructed_spectrum+bl_median_smoother, color="blue")

# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokpksallscl)

In [21]:
default_plot_height

600

In [22]:
pararaqui

NameError: name 'pararaqui' is not defined

In [ ]:
# min_snr tem que ser menor
# noise_perc tem que ser > 5 senao escapam muitos. Usar 7
ipks2 = find_peaks_cwt(y0s,widths=[widths[1]],wavelet=ricker,max_distances=widths/4,min_snr=1.0,noise_perc=10)
ipks2

In [ ]:
len(ipks2)

In [ ]:
# Separando os picos por escala:
pk_per_scal = []
for wid in widths:
    print (wid)
    pk_per_scal.append(find_peaks_cwt(y0s,widths=[wid],wavelet=ricker,max_distances=widths/4,min_snr=1.0,noise_perc=10))
    print(pk_per_scal[-1])

In [ ]:
# Bokeh graph w/ peaks in all scales
bokpksallscl = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="synth",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokpksallscl.circle(ar1[:,0], ar1[:,1], legend="cwt peaks", color="navy",size=2)
# bokpksallscl.line(xs, y0s, color="lightgreen")
# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokpksallscl)

In [ ]:
stru_diag_also = generate_binary_structure(2,2)
stru_diag_also

In [ ]:
lblimag1, num_feat_imag1 = label(imag1,stru_diag_also)
lblimag1, num_feat_imag1

In [ ]:
# Finally, the ridges delimited all over the spectrum
ridges = find_objects(lblimag1)
ridges

In [ ]:
ridges_rectangles = [lblimag1[loc] for loc in ridges]
ridges_rectangles

In [ ]:
# 2019-09-18 Parei Aqqui

In [ ]:
retang2 = ma.masked_where(lblchoice!=0,cwtchoice)
maxim2 = np.max(retang2)
maxim2

In [ ]:
retang3 = ma.masked_where(lblchoice!=3,cwtchoice)
maxim3 = np.max(retang3)
maxim3

In [ ]:
retang3

In [ ]:
np.argmax(retang2)

In [ ]:
np.argmax(retang3)

In [ ]:
np.argmax(retang3,axis=0)

In [ ]:
np.argmax(retang3,axis=1)

In [ ]:
# 2019-09-19 Fica muito confuso usar os mahximos nos eixos. Melhor usar indice flatten
# e depois converter para ihndice estrutural (unravel). Deu certo!
#                                                             |
#                                                             |
#                                                             v

In [ ]:
np.argmax(retang3)

In [ ]:
ind_retang = np.argmax(retang3)
np.unravel_index(ind_retang,np.shape(retang3))

In [ ]:
# 2019-09-22 Agora, fazendo o loop:

np.unravel_index(37,np.shape(retang3))

In [ ]:
scalmaxima = np.argmax(cwtmatr[:,pkchoice])
scalmaxima

In [ ]:
escalamaxima = widths[scalmaxima]
escalamaxima

In [ ]:
cwtmax=cwtmatr[scalmaxima,pkchoice]
cwtmax

In [ ]:
xpico = np.array(range(pkchoice-5,pkchoice+6))
xpico

In [ ]:
ypico = y0s[xpico]
ypico

In [ ]:
ypicocwt = cwtmatr[scalmaxima,xpico]
ypicocwt

In [ ]:
maxcwt = np.max(ypicocwt)
maxcwt

In [ ]:
centroide = np.average(xpico[4:7],weights=ma.masked_less(ypico[4:7],0))
centroide

In [ ]:
centroide = np.average(xpico[4:7],weights=ypico[4:7])
centroide

In [ ]:
centroidecwt1 = np.average(xpico,weights=ma.masked_less(ypicocwt,0))
centroidecwt1

In [ ]:
centroidecwt2 = np.average(xpico,weights=ypicocwt)
centroidecwt2

In [ ]:
kfwhm = 2*np.sqrt(2*np.log(2))
kfwhm

In [ ]:
# escala = sigma * raiz(5)

In [ ]:
sigmamaximo = escalamaxima / np.sqrt(5)
sigmamaximo

In [ ]:
tressigma = 3*sigmamaximo
tressigma

In [ ]:
gau_hei = k * maxcwt / np.sqrt(sigmamaximo)
gau_hei

In [ ]:
# Estah deslocado com centroidecwt1!
# ypicofit = gaus_sig(xpicofit,sigma3,gau_hei,centroidecwt1)
ypicofit = gaus_sig(xpicofit,sigmamaximo,gau_hei,centroide)
ypicofit

In [ ]:
ypicofitcanal = gaus_sig(xpico,sigmamaximo,gau_hei,centroide-0.3)
ypicofitcanal

In [ ]:
subtracao = ypico - ypicofitcanal
subtracao

In [ ]:
bokumpico = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_range=[10**1, 10**5], title="synth",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokumpico.circle(xpico, ypico, legend="counts", color="blue",size=4)
bokumpico.line(xpico, ypicofitcanal, color="lightgreen")
bokumpico.line(xpico, subtracao, color="brown")
show(bokumpico)

In [ ]:
x = np.sqrt(ipks)

In [ ]:
sigmas_experim, gau_hights

In [ ]:
len(sigmas_experim)

In [ ]:
len(gau_hights)

In [ ]:
xcp = np.copy(x)
fwhms = kfwhm * sigmas_experim
ycp = np.copy(fwhms)
sycp = np.sqrt(gau_hights)**ycp

c, stats = polyfit(xcp,ycp,1,full=True,w=sycp)
pfit = P(c)
print(pfit)
x_gr = np.linspace(xcp[0], xcp[-1])
y_gr = pfit(x_gr)
ch_gr = x_gr**2
yfit = pfit(x)

mask = [(i>3.0) & (i<12.0) for i in ycp]

xcp=xcp[mask]
ycp=ycp[mask]
sycp=sycp[mask]

c, stats = polyfit(xcp,ycp,1,full=True,w=sycp)
# c, stats = polyfit(xcp,ycp,1,full=True)
pfit = P(c)
print(pfit)
x_gr = np.linspace(xcp[0], xcp[-1])
y_gr = pfit(x_gr)
ch_gr = x_gr**2
yfit = pfit(x)

p01 = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   y_range=[0.0, 25.0], title="Plot #1: fwhms",
   x_axis_label='ch', y_axis_label='fwhm'
)
p01.circle(ipks, fwhms, legend="fwhm", color="blue",size=4)
p01.line(ch_gr, y_gr, color="green", legend="y_gr")
show(p01)

In [ ]:
pfit

In [ ]:
1 & 1

In [ ]:
# Coeficientes de a0 a an
# residuals  array...: 

# coef : ndarray, shape (deg + 1,) or (deg + 1, K)
#  Polynomial coefficients ordered from low to high. If y was 2-D, the coefficients in column k of coef
# represent the polynomial fit to the data in y’s k-th column.
# [residuals, rank, singular_values, rcond] : list
# These values are only returned if full = True
# resid – sum of squared residuals of the least squares fit
# rank – the numerical rank of the scaled Vandermonde matrix
# sv – singular values of the scaled Vandermonde matrix
# rcond – value of rcond.
#

In [ ]:
xcp

In [ ]:
ycp

In [ ]:
fitd_fwhms = pfit(xcp)
fitd_fwhms

In [ ]:
bokspec2 = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   y_axis_type="log",
   y_range=[10**1, 10**5], title="ALMERA?",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokspec.circle(xs, y0s, legend="counts", color="lightblue",size=4)
bokspec.line(xs, y0s, color="lightgreen")
#bokspec.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
#bokspec.cross(pk_per_scal[ 1], y0s[pk_per_scal[ 1]]*1.01**2, color="orange",size=4)
#bokspec.cross(pk_per_scal[ 3], y0s[pk_per_scal[ 3]]*1.01**3, color="brown",size=6)
#bokspec.cross(pk_per_scal[ 5], y0s[pk_per_scal[ 5]]*1.01**4, color="green",size=16)
#bokspec.cross(pk_per_scal[ 8], y0s[pk_per_scal[ 8]]*1.01**5, color="cyan",size=10)
#bokspec.cross(pk_per_scal[12], y0s[pk_per_scal[12]]*1.01**6, color="blue",size=12)
#bokspec.cross(pk_per_scal[15], y0s[pk_per_scal[15]]*1.01**7, color="magenta",size=14)
#bokspec.cross(pk_per_scal[20], y0s[pk_per_scal[20]]*1.01**8, color="violet",size=16)
show(bokspec2)

In [ ]:
# 2019-09-13 PAREi AQUI!!

In [ ]:
bokfwhm = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="synth",
   y_range=[0.0, 15.0], title="fwhms",
   x_axis_label='channels', y_axis_label='fwhms'
)
bokfwhm.circle(ipks, fwhms, legend="fwhms", color="blue",size=4)
bokfwhm.cross(x**x, fitd_fwhms, color="red")
# bokfwhm.line(xs, pfit[ipks], color="blue",size=8)
show(bokfwhm)

In [ ]:
# Outro teste: mahximo do cwt no espectro inteiro em cada escala

In [ ]:
maxs_escalas = np.max(cwtmatr,axis=1)
print(len(maxs_escalas))
print(maxs_escalas)

In [ ]:
print(np.argmax(maxs_escalas))
print(np.max(maxs_escalas))
widths[np.argmax(maxs_escalas)]

In [ ]:
# Entao a soma maxima eh na escala 5.10, entre 4.48 e 5.72. Interessante. Visualmente, achei melhor pegar
# essa mesmo.
# 2019-09-17 Era, jah naum eh mais...
# Agora eh 5.71, entre 5.45 e 5.98